In [1]:

!pip install instructor 

  Using cached rich-13.7.0-py3-none-any.whl.metadata (18 kB)
Using cached rich-13.7.0-py3-none-any.whl (240 kB)
  Attempting uninstall: rich
    Found existing installation: rich 13.5.3
    Uninstalling rich-13.5.3:
      Successfully uninstalled rich-13.5.3


In [2]:
import instructor
from openai import OpenAI
from pydantic import BaseModel

In [3]:
client = instructor.patch(OpenAI())

In [6]:
class UserDetail(BaseModel):
    name: str
    age: int
    
def extract(data: str) -> UserDetail:
    return client.chat.completions.create(
        model="gpt-4-1106-preview",
        response_model=UserDetail,
        messages=[
            {"role": "user", "content": data}
        ]
    )

In [8]:
import functools

@functools.cache
def extract(data):  # noqa: F811
    return client.chat.completions.create(
        model="gpt-4-1106-preview",
        response_model=UserDetail,
        messages=[
            {"role": "user", "content": data}
        ]
    )

In [11]:
import time


start = time.perf_counter()
model = extract("Extract jason is 25 years old")
print(f"Time taken: {time.perf_counter() - start}")

start = time.perf_counter()
model = extract("Extract jason is 25 years old")
print(f"Time taken: {time.perf_counter() - start}")

start = time.perf_counter()
model = extract("Extract jason is 254 years old")
print(f"Time taken: {time.perf_counter() - start}")

Time taken: 2.4124994524754584e-05
Time taken: 1.7708996892906725e-05
Time taken: 1.6999998479150236e-05


In [13]:
!pip install diskcache

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00


In [14]:
import functools
import inspect
import instructor
import diskcache

from openai import OpenAI
from pydantic import BaseModel

In [15]:
client = instructor.patch(OpenAI())
cache = diskcache.Cache("cache")

In [19]:
def instructor_cache(func):
    """Cache a function that returns a Pydantic model"""
    return_type = inspect.signature(func).return_annotation
    if not issubclass(return_type, BaseModel):
        raise ValueError("The return type must be a Pydantic model")
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        key = f"{func.__name__}-{functools._make_key(args, kwargs, typed=False)}"
        # Check if the result is already cached
        if (cached := cache.get(key)) is not None:
            return return_type.model_validate_json(cached)
        
        # Call the function and cache its result
        result = func(*args, **kwargs)
        serialized_result = result.model_dump_json()
        cache.set(key, serialized_result)
        
        return result
    
    return wrapper


class UserDetail(BaseModel):
    name: str
    age: int

In [20]:
@instructor_cache
def extract(data) -> UserDetail:
    return client.chat.completions.create(
        model="gpt-4-1106-preview",
        response_model=UserDetail,
        messages=[
            {"role": "user", "content": data}
        ]
    )

In [27]:
import redis
import functools
import inspect
import json
import instructor

from openai import OpenAI
from pydantic import BaseModel

client = instructor.patch(OpenAI())
cache = redis.Redis(host="localhost", port=6379, db=0, password="123456")

In [31]:
def instructor_redis(func):
    """Cache a function that returns a Pydantic model"""
    return_type = inspect.signature(func).return_annotation
    if not issubclass(return_type, BaseModel):
        raise ValueError("The return type must be a Pydantic model")
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        key = f"{func.__name__}-{functools._make_key(args, kwargs, typed=False)}"
        # Check if the result is already cached
        if (cached := cache.get(key)) is not None:
            return return_type.model_validate(json.loads(cached))
        
        # Call the function and cache its result
        result = func(*args, **kwargs)
        serialized_result = result.model_dump_json()
        cache.set(key, serialized_result)
        
        return result
    return wrapper


class UserDetail(BaseModel):
    name: str
    age: int
    
@instructor_redis
def extract_redis(data) -> UserDetail:
    return client.chat.completions.create(
        model="gpt-4-1106-preview",
        response_model=UserDetail,
        messages=[
            {"role": "user", "content": data}
        ]
    )

In [34]:
import time


start = time.perf_counter()
model = extract_redis("Extract jason is 25 years old")
print(f"Time taken: {time.perf_counter() - start}")

start = time.perf_counter()
model = extract_redis("Extract jason is 25 years old")
print(f"Time taken: {time.perf_counter() - start}")

start = time.perf_counter()
model = extract_redis("Extract jason is 254 years old")
print(f"Time taken: {time.perf_counter() - start}") 

Time taken: 0.00898504200449679
Time taken: 0.002607749993330799
Time taken: 0.003841083002043888
